In [1]:
import torch
import numpy as np
from torchvision import transforms
import torchvision
from torch.utils.tensorboard import SummaryWriter
import datasets as mt_datasets
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
import imageio


In [2]:
evice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

writer = SummaryWriter()

nodules = torch.load('nodules.pt')
malignancies = torch.load('malignancies.pt') - 1

batch_size = 32
validation_split = .1
shuffle_dataset = True
random_seed= 42


In [3]:
dataset = mt_datasets.TensorDataset(nodules, malignancies)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


In [5]:
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 5)
# resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

In [7]:
print(len(train_loader))

15
